## Ideas:

### Attributes

- illumination (one or two) - how does Flamingo converter check these attributes?
- channels
- time points
- positions
- frame interval - diff between last and first time stamp.
- magnification - probably just parse metadata file.

### Methods

- Save max projection
    - Memory converving option: open/save individual frames in dir
    - Full stack option: concatenate MPs into stack and save stack.
- Downsample 3D
    - Memory converving option: open/save individual 3D frames in dir
    - Full stack option: concatenate MPs into stack and save stack.
    - View in Napari True/False
- Combined Max projection and Downsample 3D
    - Computes both at once as a nice saver. Will use slightly more memory but save a lot of time. 
    - 

### Thoughts

- it would probably be faster to 1) max project each side, then max project those together and 2) downsample 3d volumes before merging instead of the current workflow which is to merge the full volume before max projecting and merging
- measure the time of each step (reading, processing, writing); see if it's worth reading in a separate thread from processing. Might need both multithreading and multiiprocessing I suppose. 

In [30]:
import os
import numpy as np

class Kkpo:

    def __init__(self, file_path):

        self.file_path = file_path

    def get_dimensions(self):
        files = [file for file in os.listdir(self.file_path) if file.endswith('.tif') and not file.startswith('.') and not 'MP' in file]
        sample_names = [file.split('_')[0] for file in files]
        timepoint_names = [file.split('_')[1] for file in files]
        view_names = [file.split('_')[2] for file in files]
        region_names = [file.split('_')[3] for file in files]
        tileX_names = [file.split('_')[4] for file in files]
        tileY_names = [file.split('_')[5] for file in files]
        channel_names = [file.split('_')[6] for file in files]
        illum_names = [file.split('_')[7] for file in files]

        ch_640 = [file for file in files if 'C00' in file]
        ch_561 = [file for file in files if 'C01' in file]
        ch_488 = [file for file in files if 'C02' in file]
        ch_405 = [file for file in files if 'C03' in file]
        trans = [file for file in files if 'C04' in file]

        print(f'unique sample names: {np.unique(sample_names)}')
        #print(f'unique time point names: {np.unique(timepoint_names)}')
        print(f'unique view names: {np.unique(view_names)}')
        print(f'unique region names: {np.unique(region_names)}')
        print(f'unique tileX names: {np.unique(tileX_names)}')
        print(f'unique tileY names: {np.unique(tileY_names)}')
        print(f'unique chanel names: {np.unique(channel_names)}')
        print(f'unique illumination names: {np.unique(illum_names)}')


        num_channels = 0
        for ch in [ch_640, ch_561, ch_488, ch_405, trans]:
            if len(ch) > 0:
                num_channels += 1

        print('***'*8)
        print(f'num channels is {num_channels}')
        print(f'num trans images is {len(trans)}')
        print(f'num 405 images is {len(ch_405)}')
        print(f'num 488 images is {len(ch_488)}')
        print(f'num 561 images is {len(ch_561)}')
        print(f'num 640 images is {len(ch_640)}')
        print('***'*8)
    


kkpo = Kkpo(file_path = '/Volumes/FlashSSD2/20220415_143603_GFPwGBDE01T01')
print(vars(kkpo))
kkpo.get_dimensions()

{'file_path': '/Volumes/FlashSSD2/20220415_143603_GFPwGBDE01T01'}
unique sample names: ['S000']
unique view names: ['V000']
unique region names: ['R0000']
unique tileX names: ['X000']
unique tileY names: ['Y000']
unique chanel names: ['C00' 'C01' 'C02']
unique illumination names: ['I0' 'I1']
************************
num channels is 3
num trans images is 0
num 405 images is 0
num 488 images is 368
num 561 images is 369
num 640 images is 370
************************


In [16]:
class Test(object):
    def __init__(self):
        self.hi()

    def hi(self):
        print ("Hi!")

In [17]:
test = Test()

Hi!


In [ ]:
file = 'S000_t000000_V000_R0000_X000_Y000_C00_I0_D0_P00135.tif'

